In [ ]:
#Installing Necessary Libraries
!pip3 install deepctr==0.7.4
!pip3 install flwr[simulation]
!pip install ray==1.11
!pip install tensorflow==2.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 81 kB 12 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 106 kB 6.1 MB/s 
     |████████████████████████████████| 15.3 MB 2.7 MB/s 
     |████████████████████████████████| 1.0 MB 48.3 MB/s 
     |████████████████████████████████| 52.7 MB 1.4 MB/s 
     |████████████████████████████████| 4.1 MB 35.7 MB/s 
     |████████████████████████████████| 244 kB 45.3 MB/s 
     |████████████████████████████████| 201 kB 21.3 MB/s 
     |████████████████████████████████| 65 kB 3.2 MB/s 
     |████████████████████████████████| 144 kB 46.4 MB/s 
     |████████████████████████████████| 3.1 MB 29.3 MB/s 
     |████████████████████████████████| 1.1 MB 42.1 MB/s 
     |████████████████████████████████| 128 kB 50.5 MB/s 
     |████████████████████████████████| 57 kB 5.2 MB/s

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 52.7 MB 1.5 MB/s 
  Attempting uninstall: ray
    Found existing installation: ray 1.11.1
    Uninstalling ray-1.11.1:
      Successfully uninstalled ray-1.11.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 421.8 MB 23 kB/s 
     |████████████████████████████████| 50 kB 6.3 MB/s 
     |████████████████████████████████| 3.8 MB 39.6 MB/s 
     |████████████████████████████████| 448 kB 73.2 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=d0a4bc88dbe4786ba47b9e70a903a5a9578e666bb80b0c6b0f8851ddca418757
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estima

In [ ]:
#Importing Required Libraries
from deepctr.models import DIN, DeepFM, AutoInt
from deepctr.inputs import SparseFeat,VarLenSparseFeat,DenseFeat,get_feature_names
import os
import math
import pandas as pd
import numpy as np
import flwr as fl
from sklearn.metrics import log_loss, roc_auc_score
import pickle
from sklearn.preprocessing import MinMaxScaler,LabelEncoder
import warnings
import ast
import plotly.express as px
import plotly.graph_objects as go
import tensorflow as tf
from tensorflow.keras import backend as K
from sklearn.metrics import log_loss, roc_auc_score
from tensorflow.keras.losses import binary_crossentropy
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
import pickle
warnings.filterwarnings('ignore')

DeepCTR version 0.9.1 detected. Your version is 0.7.4.
Use `pip install -U deepctr` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR/releases/tag/v0.9.1


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# !rm -r drive/MyDrive/avazu
# !rm -r eval_logs
# !rm -r train_logs

In [ ]:
#Making Required File Directories
# !mkdir train_logs
# !mkdir eval_logs
# !mkdir drive/MyDrive/avazu/

In [ ]:
#Setting Up Parameters
FILE_PATH = "drive/MyDrive/200ksample.csv"
TOTAL_ROUNDS = 100
ROUND_PER_LOOP = 2
NUM_CLIENTS = 100
FRACTION_FIT = 0.2 # Sample 10% of available clients for training
FRACTION_EVAL = 0.001  # Sample 5% of available clients for evaluation
MIN_FIT_CLIENTS = 20 # Never sample less than 10 clients for training
MIN_EVAL_CLIENTS = 1  # Never sample less than 5 clients for evaluation
MIN_AVAILABLE_CLIENTS=int(NUM_CLIENTS * 0.05)

In [ ]:
#Data Loading And Pre-Processing
data = pd.read_csv(FILE_PATH)
data = data.drop(columns=['Unnamed: 0'])
data.set_index('hour',inplace=True)
sparse_features = data.iloc[::,2:14].columns.values.tolist()
dense_features = data.iloc[::,15:].columns.values.tolist()
# sparse_features.append('id')
target = ['click']

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200000 entries, 2014-10-21 00:00:00 to 2014-10-30 23:00:00
Data columns (total 23 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id                200000 non-null  uint64
 1   click             200000 non-null  int64 
 2   C1                200000 non-null  int64 
 3   banner_pos        200000 non-null  int64 
 4   site_id           200000 non-null  object
 5   site_domain       200000 non-null  object
 6   site_category     200000 non-null  object
 7   app_id            200000 non-null  object
 8   app_domain        200000 non-null  object
 9   app_category      200000 non-null  object
 10  device_id         200000 non-null  object
 11  device_ip         200000 non-null  object
 12  device_model      200000 non-null  object
 13  device_type       200000 non-null  int64 
 14  device_conn_type  200000 non-null  int64 
 15  C14               200000 non-null  int64 
 16  C15         

In [ ]:
def Mixed_loss(a = 1.0):
    """
    """
#     a = tf.constant(a, dtype=tf.float32)

    def binary_focal_loss_fixed(y_true, y_pred):
        alpha = tf.constant(0.8, dtype=tf.float32)
        gamma = tf.constant(0.6, dtype=tf.float32)
        """
        y_true shape need be (None,1)
        y_pred need be compute after sigmoid
        """
        y_true = tf.cast(y_true, tf.float32)
        alpha_t = y_true*alpha + (K.ones_like(y_true)-y_true)*(1-alpha)
    
        p_t = y_true*y_pred + (K.ones_like(y_true)-y_true)*(K.ones_like(y_true)-y_pred) + K.epsilon()
        focal_loss = - alpha_t * K.pow((K.ones_like(y_true)-p_t),gamma) * K.log(p_t)
#         return K.mean(focal_loss)
        return focal_loss
        
    def mixed_loss(y_true, y_pred):
        a = tf.constant(0, dtype=tf.float32)
#         return a*binary_focal_loss_fixed(y_true, y_pred)+(1-a)*binary_crossentropy(y_true,y_pred)
        return binary_crossentropy(y_true,y_pred)
    return mixed_loss


In [ ]:
scaler = MinMaxScaler(feature_range=(0,1))
encoder = LabelEncoder()
for feat in sparse_features:
    data[feat] = encoder.fit_transform(data[feat])
data[dense_features] = scaler.fit_transform(data[dense_features])
y = data[target]
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size=0.2, random_state=42)
x_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

fixlen_feature_columns = [SparseFeat(feat,data[feat].nunique()) 
                          for feat in sparse_features] + [DenseFeat(feat,1,)
                                                         for feat in dense_features]
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns
feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)
# pickle.dump(dnn_feature_columns,open('drive/MyDrive/dnn_feature_columns.pkl','wb'))
# pickle.dump(linear_feature_columns,open('drive/MyDrive/linear_feature_columns.pkl','wb'))
# pickle.dump(feature_names,open('drive/MyDrive/feature_names.pkl','wb'))
y_test = X_test['click'].values
test_model_input = {name: X_test[name] for name in feature_names}
y_val = x_val['click'].values
x_val = {name:x_val[name] for name in feature_names}
users = data['device_id'].unique()

In [ ]:
#Flower Client Setup
class FlowerClient(fl.client.NumPyClient):
    def __init__(self,model,x_train,y_train,x_val,y_val,cid):
        self.model = model
        self.x_train , self.y_train = x_train, y_train
        self.x_val, self.y_val = x_val, y_val
        self.cid = cid
    def get_parameters(self):
        return self.model.get_weights()
    def get_properties(self):
        return None
    def fit(self,parameters, config):
        if config['rnd'] > 1 and config['rnd']%2==1: #Change %x by number set at ROUND_PER_LOOP
          # print(f"Loading Weights From Rnd {config['rnd']-1}")
          self.model.load_weights(f"drive/MyDrive/avazu/round_{config['rnd']-1}/saved_model{config['rnd']-1}")
        else:
          self.model.set_weights(parameters)
        try:
          history = self.model.fit(self.x_train, self.y_train,batch_size=1024, epochs=2, verbose=0,validation_data=[x_val,y_val])
          # with open("train_logs/train_log_{}".format(self.cid),"a+") as f:
          #   f.write(str(history.history))
        except Exception as e:
          print(e)
        return self.model.get_weights(), len(self.x_train), {}
    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        if not os.path.exists(f"drive/MyDrive/avazu/round_{config['rnd']}"):
          self.model.save_weights(f"drive/MyDrive/avazu/round_{config['rnd']}/saved_model{config['rnd']}")       
        # loss = self.model.evaluate(self.x_val, self.y_val,batch_size = 4096)
        # round_val_auc.append(loss[2])
        # round_val_loss.append(loss[0])
        # with open("eval_logs/eval_log_{}".format(self.cid),"a+") as f:
        #     f.write(str(loss))
        return 1
        # return loss[0], len(self.x_val), {"auc": loss[1]}
def client_fn(cid:str) -> fl.client.Client:
    model = AutoInt(linear_feature_columns,dnn_feature_columns,
               task='binary',dnn_dropout=0.5)
    model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['binary_crossentropy','AUC'])
    idx = len(X_train)//NUM_CLIENTS
    s = (int(cid)-1)*idx
    f = s+idx
    x_train = X_train.iloc[s:f]
    y_train_cid = x_train['click'].values
    x_train_cid = {name:x_train[name] for name in feature_names}
    # print("Created client {} with data length {}".format(cid,len(y_train_cid)))
    # Create and return client
    return FlowerClient(model, x_train_cid, y_train_cid, x_val, y_val,int(cid))  

In [ ]:
#Overloading Function
global_rnd = 1
def evaluate_config(rnd: int):
    """Return evaluation configuration dict for each round.
    Perform five local evaluation steps on each client (i.e., use five
    batches) during rounds one to three, then increase to ten local
    evaluation steps.
    """
    global global_rnd
    global_rnd +=1
    return {"rnd": global_rnd-1}
def fit_config(rnd: int):
  global global_rnd
  return {"rnd":global_rnd}

In [ ]:
#Defining Server Strategy
strategy=fl.server.strategy.FedAvg(
       fraction_fit=FRACTION_FIT,  # Sample 10% of available clients for training
        fraction_eval=FRACTION_EVAL,  # Sample 5% of available clients for evaluation
       min_fit_clients=MIN_FIT_CLIENTS,  # Never sample less than 10 clients for training
        min_eval_clients=MIN_EVAL_CLIENTS,  # Never sample less than 5 clients for evaluation
        min_available_clients=MIN_AVAILABLE_CLIENTS, # Wait until at least 75 clients are available
        on_evaluate_config_fn=evaluate_config,
        on_fit_config_fn = fit_config,
)


WARNING flower 2022-06-20 19:40:28,322 | fedavg.py:109 | 
Setting `min_available_clients` lower than `min_fit_clients` or
`min_eval_clients` can cause the server to fail when there are too few clients
connected to the server. `min_available_clients` must be set to a value larger
than or equal to the values of `min_fit_clients` and `min_eval_clients`.

Setting `min_available_clients` lower than `min_fit_clients` or
`min_eval_clients` can cause the server to fail when there are too few clients
connected to the server. `min_available_clients` must be set to a value larger
than or equal to the values of `min_fit_clients` and `min_eval_clients`.



In [ ]:
#Starting The Server and Simulation Process
for i in range(0,TOTAL_ROUNDS,ROUND_PER_LOOP):
  print(f"\n Starting For Loop : {i}")
  fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    num_rounds=ROUND_PER_LOOP,
    strategy=strategy,)
  
  


 Starting For Loop : 60


INFO flower 2022-06-20 06:17:56,379 | app.py:158 | Ray initialized with resources: {'node:172.28.0.2': 1.0, 'object_store_memory': 3059680051.0, 'CPU': 2.0, 'memory': 6119360103.0}
INFO:flower:Ray initialized with resources: {'node:172.28.0.2': 1.0, 'object_store_memory': 3059680051.0, 'CPU': 2.0, 'memory': 6119360103.0}
INFO flower 2022-06-20 06:17:56,393 | app.py:174 | Starting Flower simulation running: Config(num_rounds=2, round_timeout=None)
INFO:flower:Starting Flower simulation running: Config(num_rounds=2, round_timeout=None)
INFO flower 2022-06-20 06:17:56,402 | server.py:84 | Initializing global parameters
INFO:flower:Initializing global parameters
INFO flower 2022-06-20 06:17:56,409 | server.py:256 | Requesting initial parameters from one random client
INFO:flower:Requesting initial parameters from one random client
(launch_and_get_parameters pid=13252) WARNING:root:
(launch_and_get_parameters pid=13252) DeepCTR version 0.9.1 detected. Your version is 0.7.4.
(launch_and_get_

(launch_and_fit pid=13252) Empty training data.


(launch_and_fit pid=13252) 2022-06-20 06:22:10.188669: W tensorflow/core/common_runtime/base_collective_executor.cc:217] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
(launch_and_fit pid=13252) 	 [[{{node IteratorGetNext}}]]
(launch_and_fit pid=13252) WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 0 batches). You may need to use the repeat() function when building your dataset.
(launch_and_fit pid=13252) /usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
(launch_and_fit pid=13252)   "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
(launch_and_fit pid=13252) /usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:4


 Starting For Loop : 62


...

INFO flower 2022-06-20 07:57:13,340 | app.py:158 | Ray initialized with resources: {'object_store_memory': 3643441152.0, 'node:172.28.0.2': 1.0, 'CPU': 2.0, 'memory': 7286882304.0}
INFO:flower:Ray initialized with resources: {'object_store_memory': 3643441152.0, 'node:172.28.0.2': 1.0, 'CPU': 2.0, 'memory': 7286882304.0}
INFO flower 2022-06-20 07:57:13,348 | app.py:174 | Starting Flower simulation running: Config(num_rounds=2, round_timeout=None)
INFO:flower:Starting Flower simulation running: Config(num_rounds=2, round_timeout=None)
INFO flower 2022-06-20 07:57:13,368 | server.py:84 | Initializing global parameters
INFO:flower:Initializing global parameters
INFO flower 2022-06-20 07:57:13,381 | server.py:256 | Requesting initial parameters from one random client
INFO:flower:Requesting initial parameters from one random client
(launch_and_get_parameters pid=47666) WARNING:root:
(launch_and_get_parameters pid=47666) DeepCTR version 0.9.1 detected. Your version is 0.7.4.
(launch_and_get_

(launch_and_fit pid=47665) Empty training data.


(launch_and_fit pid=47665) 2022-06-20 08:01:53.015286: W tensorflow/core/common_runtime/base_collective_executor.cc:217] BaseCollectiveExecutor::StartAbort Out of range: End of sequence
(launch_and_fit pid=47665) 	 [[{{node IteratorGetNext}}]]
(launch_and_fit pid=47665) WARNING:tensorflow:Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches (in this case, 0 batches). You may need to use the repeat() function when building your dataset.
(launch_and_fit pid=47665) /usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
(launch_and_fit pid=47665)   "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
(launch_and_fit pid=47665) /usr/local/lib/python3.7/dist-packages/tensorflow_core/python/framework/indexed_slices.py:4


 Starting For Loop : 98


INFO flower 2022-06-20 08:02:37,569 | app.py:158 | Ray initialized with resources: {'object_store_memory': 3643497676.0, 'node:172.28.0.2': 1.0, 'CPU': 2.0, 'memory': 7286995355.0}
INFO:flower:Ray initialized with resources: {'object_store_memory': 3643497676.0, 'node:172.28.0.2': 1.0, 'CPU': 2.0, 'memory': 7286995355.0}
INFO flower 2022-06-20 08:02:37,582 | app.py:174 | Starting Flower simulation running: Config(num_rounds=2, round_timeout=None)
INFO:flower:Starting Flower simulation running: Config(num_rounds=2, round_timeout=None)
INFO flower 2022-06-20 08:02:37,599 | server.py:84 | Initializing global parameters
INFO:flower:Initializing global parameters
INFO flower 2022-06-20 08:02:37,631 | server.py:256 | Requesting initial parameters from one random client
INFO:flower:Requesting initial parameters from one random client
(launch_and_get_parameters pid=49522) WARNING:root:
(launch_and_get_parameters pid=49522) DeepCTR version 0.9.1 detected. Your version is 0.7.4.
(launch_and_get_

In [ ]:
def plot_vals(x,y,title):
  fig = px.line([i for i in range(1,x+1)],y= y,title=title)
  fig.show()

In [ ]:
#User Level AUC Score
model = AutoInt(linear_feature_columns,dnn_feature_columns,
               task='binary',dnn_dropout=0.5)
model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['AUC','FalsePositives','FalseNegatives','TruePositives','TrueNegatives'])
round_val_auc = []
round_val_loss = []
for j in range(1,TOTAL_ROUNDS+1):
  print(f"Eval For Round : {j}")
  varscore = 0
  lossvar = 0
  model.load_weights('drive/MyDrive/avazu/round_{}/saved_model{}'.format(j,j))
  try:
    loss = model.evaluate(x_val, y_val,batch_size=4096,verbose=0)
    varscore = loss[1]
    lossvar = loss[0]
    print(f"AUC: {varscore},Loss: {lossvar},TP: {loss[4]},TN: {loss[5]}, FP: {loss[2]},FN: {loss[3]}")
  except Exception as e:
    print(e)
  round_val_auc.append(varscore)
  round_val_loss.append(lossvar)

Eval For Round : 1
AUC: 0.52437424659729,Loss: 0.4781121024131775,TP: 0.0,TN: 16567.0, FP: 0.0,FN: 3433.0
Eval For Round : 2
AUC: 0.549286961555481,Loss: 0.46379222702980044,TP: 0.0,TN: 16567.0, FP: 0.0,FN: 3433.0
Eval For Round : 3
AUC: 0.5692750811576843,Loss: 0.456368359375,TP: 0.0,TN: 16567.0, FP: 0.0,FN: 3433.0
Eval For Round : 4
AUC: 0.5923680663108826,Loss: 0.451532154083252,TP: 0.0,TN: 16567.0, FP: 0.0,FN: 3433.0
Eval For Round : 5
AUC: 0.6105539798736572,Loss: 0.4483344175338745,TP: 0.0,TN: 16567.0, FP: 0.0,FN: 3433.0
Eval For Round : 6
AUC: 0.6238740086555481,Loss: 0.4460032935142517,TP: 0.0,TN: 16567.0, FP: 0.0,FN: 3433.0
Eval For Round : 7
AUC: 0.6324858069419861,Loss: 0.44411195154190064,TP: 0.0,TN: 16567.0, FP: 0.0,FN: 3433.0
Eval For Round : 8
AUC: 0.639041543006897,Loss: 0.44230029010772703,TP: 0.0,TN: 16567.0, FP: 0.0,FN: 3433.0
Eval For Round : 9
AUC: 0.644079864025116,Loss: 0.4407668894767761,TP: 0.0,TN: 16567.0, FP: 0.0,FN: 3433.0
Eval For Round : 10
AUC: 0.65054529

In [ ]:
plot_vals(TOTAL_ROUNDS,round_val_auc,"AUC Score")
plot_vals(TOTAL_ROUNDS,round_val_loss,"Loss")

In [ ]:
#User Level AUC Score
model = AutoInt(linear_feature_columns,dnn_feature_columns,
               task='binary',dnn_dropout=0.5)
model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['AUC','FalsePositives','FalseNegatives','TruePositives','TrueNegatives'])
rounds_auc_user_avg = []
rounds_loss_user_avg = []
rounds_true_pos = []
rounds_true_neg = []
rounds_false_pos = []
rounds_false_neg = []
for j in range(1,TOTAL_ROUNDS+1):
  print(f"Eval For Round : {j}")
  varscore = 0
  lossvar = 0
  tp = 0
  tn = 0
  fp = 0
  fn = 0
  model.load_weights('drive/MyDrive/avazu/round_{}/saved_model{}'.format(j,j))
  try:
    loss = model.evaluate(test_model_input, y_test,batch_size=4096,verbose=0)
    varscore = loss[1]
    lossvar = loss[0]
    print(f"AUC: {varscore},Loss: {lossvar},TP: {loss[4]},TN: {loss[5]}, FP: {loss[2]},FN: {loss[3]}")
  except Exception as e:
    print(e)
  rounds_auc_user_avg.append(varscore)
  rounds_loss_user_avg.append(lossvar)
  rounds_true_pos.append(loss[4])
  rounds_true_neg.append(loss[5])
  rounds_false_pos.append(loss[2])
  rounds_false_neg.append(loss[3]) 

Eval For Round : 1
AUC: 0.5153989791870117,Loss: 0.4774716869354248,TP: 0.0,TN: 16604.0, FP: 0.0,FN: 3396.0
Eval For Round : 2
AUC: 0.5395777225494385,Loss: 0.46289758915901186,TP: 0.0,TN: 16604.0, FP: 0.0,FN: 3396.0
Eval For Round : 3
AUC: 0.5591830611228943,Loss: 0.45536065020561217,TP: 0.0,TN: 16604.0, FP: 0.0,FN: 3396.0
Eval For Round : 4
AUC: 0.5823708772659302,Loss: 0.45037936525344846,TP: 0.0,TN: 16604.0, FP: 0.0,FN: 3396.0
Eval For Round : 5
AUC: 0.6038275957107544,Loss: 0.4470381872653961,TP: 0.0,TN: 16604.0, FP: 0.0,FN: 3396.0
Eval For Round : 6
AUC: 0.6185809373855591,Loss: 0.4444432110309601,TP: 0.0,TN: 16604.0, FP: 0.0,FN: 3396.0
Eval For Round : 7
AUC: 0.6282045841217041,Loss: 0.4422906684398651,TP: 0.0,TN: 16604.0, FP: 0.0,FN: 3396.0
Eval For Round : 8
AUC: 0.6366302371025085,Loss: 0.44017969727516176,TP: 0.0,TN: 16604.0, FP: 0.0,FN: 3396.0
Eval For Round : 9
AUC: 0.6425002217292786,Loss: 0.43835046162605285,TP: 0.0,TN: 16604.0, FP: 0.0,FN: 3396.0
Eval For Round : 10
AUC

In [ ]:
plot_vals(TOTAL_ROUNDS,rounds_auc_user_avg,"AUC Score")
plot_vals(TOTAL_ROUNDS,rounds_loss_user_avg,"Loss")
plot_vals(TOTAL_ROUNDS,rounds_true_pos,"rounds_true_pos")
plot_vals(TOTAL_ROUNDS,rounds_true_neg,"rounds_true_neg")
plot_vals(TOTAL_ROUNDS,rounds_false_pos,"rounds_false_pos")
plot_vals(TOTAL_ROUNDS,rounds_false_neg,"rounds_false_neg")


In [ ]:
model = AutoInt(linear_feature_columns,dnn_feature_columns,
               task='binary',dnn_dropout=0.5)
model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['AUC','FalsePositives','FalseNegatives','TruePositives','TrueNegatives'])
train_auc = []
train_loss = []
train_model_input = {name:X_train[name] for name in feature_names}
for j in range(1,TOTAL_ROUNDS+1):
  print(f"Eval For Round : {j}")
  varscore = 0
  lossvar = 0
  model.load_weights('drive/MyDrive/avazu/round_{}/saved_model{}'.format(j,j))
  try:
    loss = model.evaluate(train_model_input, y_train,batch_size=4096,verbose=0)
    varscore = loss[1]
    lossvar = loss[0]
    print(f"AUC: {varscore},Loss: {lossvar},TP: {loss[4]},TN: {loss[5]}, FP: {loss[2]},FN: {loss[3]}")
  except Exception as e:
    print(e)
  train_auc.append(varscore)
  train_loss.append(lossvar)

Eval For Round : 1
AUC: 0.5172480344772339,Loss: 0.47471890869140626,TP: 0.0,TN: 133145.0, FP: 0.0,FN: 26855.0
Eval For Round : 2
AUC: 0.5421857237815857,Loss: 0.45933503937721254,TP: 0.0,TN: 133145.0, FP: 0.0,FN: 26855.0
Eval For Round : 3
AUC: 0.561881422996521,Loss: 0.45191054611206055,TP: 0.0,TN: 133145.0, FP: 0.0,FN: 26855.0
Eval For Round : 4
AUC: 0.5854827165603638,Loss: 0.44706381130218503,TP: 0.0,TN: 133145.0, FP: 0.0,FN: 26855.0
Eval For Round : 5
AUC: 0.6052754521369934,Loss: 0.44384197940826414,TP: 0.0,TN: 133145.0, FP: 0.0,FN: 26855.0
Eval For Round : 6
AUC: 0.6189246773719788,Loss: 0.44138774909973144,TP: 0.0,TN: 133145.0, FP: 0.0,FN: 26855.0
Eval For Round : 7
AUC: 0.628176212310791,Loss: 0.4393663149356842,TP: 0.0,TN: 133145.0, FP: 0.0,FN: 26855.0
Eval For Round : 8
AUC: 0.6361732482910156,Loss: 0.43749199023246765,TP: 0.0,TN: 133145.0, FP: 0.0,FN: 26855.0
Eval For Round : 9
AUC: 0.6415438652038574,Loss: 0.4357428717136383,TP: 0.0,TN: 133145.0, FP: 0.0,FN: 26855.0
Eval 

In [ ]:
plot_vals(TOTAL_ROUNDS,train_auc,"AUC Score")
plot_vals(TOTAL_ROUNDS,train_loss,"Loss")

In [ ]:
# auc = {"train":train_auc,"val":round_val_auc,"test":rounds_auc_user_avg}
fig = go.Figure()
fig.add_trace(go.Scatter(x=[i for i in range(1,TOTAL_ROUNDS+1)], y=train_auc,
                    mode='lines',
                    name='TRAIN AUC'))
fig.add_trace(go.Scatter(x=[i for i in range(1,TOTAL_ROUNDS+1)], y=round_val_auc,
                    mode='lines',
                    name='VAL AUC'))
fig.add_trace(go.Scatter(x=[i for i in range(1,TOTAL_ROUNDS+1)], y=rounds_auc_user_avg,
                    mode='lines',
                    name='TEST AUC'))
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=[i for i in range(1,TOTAL_ROUNDS+1)], y=train_loss,
                    mode='lines',
                    name='TRAIN LOSS'))
fig.add_trace(go.Scatter(x=[i for i in range(1,TOTAL_ROUNDS+1)], y=round_val_loss,
                    mode='lines',
                    name='VAL LOSS'))
fig.add_trace(go.Scatter(x=[i for i in range(1,TOTAL_ROUNDS+1)], y=rounds_loss_user_avg,
                    mode='lines',
                    name='TEST LOSS'))
fig.show()

In [ ]:
print(train_loss)
print(round_val_loss)
print(rounds_loss_user_avg)
print(train_auc)
print(round_val_auc)
print(rounds_auc_user_avg)


[0.47471890869140626, 0.45933503937721254, 0.45191054611206055, 0.44706381130218503, 0.44384197940826414, 0.44138774909973144, 0.4393663149356842, 0.43749199023246765, 0.4357428717136383, 0.43407392630577085, 0.4324373745918274, 0.43091747136116026, 0.42944961924552916, 0.4278727382183075, 0.4262581744670868, 0.4247138774394989, 0.4229962733268738, 0.4216325581073761, 0.42023342232704164, 0.4187789542198181, 0.41740368723869326, 0.4160254861831665, 0.4149288161277771, 0.41408221888542174, 0.41317824802398684, 0.4124322004318237, 0.41174537477493284, 0.4112931337833405, 0.41076324062347414, 0.41039819831848146, 0.4102805901527405, 0.4098808511734009, 0.4096204180240631, 0.4094522511959076, 0.40928319873809815, 0.409182008934021, 0.4090389958381653, 0.4089897649288177, 0.4088135678291321, 0.4087381363391876, 0.40872506008148196, 0.40872339844703676, 0.40856402707099915, 0.40866189279556275, 0.40840030326843263, 0.4084196773529053, 0.40827814660072326, 0.40831294717788696, 0.4083123898506

In [ ]:
auc_scores = [0.51,0.536,0.54,0.54,0.746,0.753,0.76,0.77]
log_loss = [6.1,5.45,5.876,5.46,0.397,0.383,0.371,0.382]
model_score_names = ['Gradient Boost','CatBoost','LightGBM','Logistic Regression','ONN','xDeepFM','Ensembler','AutoInt(Baseline)']
fig = go.Figure()
fig.add_trace(go.Scatter(x=[i for i in range(1,TOTAL_ROUNDS+1)], y=train_auc,
                    mode='lines',
                    name='TRAIN AUC'))
fig.add_trace(go.Scatter(x=[i for i in range(1,TOTAL_ROUNDS+1)], y=round_val_auc,
                    mode='lines',
                    name='VAL AUC'))
fig.add_trace(go.Scatter(x=[i for i in range(1,TOTAL_ROUNDS+1)], y=rounds_auc_user_avg,
                    mode='lines',
                    name='TEST AUC'))

for i in range(len(auc_scores)):
  fig.add_trace(go.Scatter(x=[i for i in range(1,TOTAL_ROUNDS+1)], y=[auc_scores[i]]*100,
                    mode='lines',
                    name=model_score_names[i]))
fig.show()

In [ ]:

fig = go.Figure()
fig.add_trace(go.Scatter(x=[i for i in range(1,TOTAL_ROUNDS+1)], y=train_loss,
                    mode='lines',
                    name='TRAIN LOSS'))
fig.add_trace(go.Scatter(x=[i for i in range(1,TOTAL_ROUNDS+1)], y=round_val_loss,
                    mode='lines',
                    name='VAL LOSS'))
fig.add_trace(go.Scatter(x=[i for i in range(1,TOTAL_ROUNDS+1)], y=rounds_loss_user_avg,
                    mode='lines',
                    name='TEST LOSS'))
for i in range(4,8):
  fig.add_trace(go.Scatter(x=[i for i in range(1,TOTAL_ROUNDS+1)], y=[log_loss[i]]*100,
                    mode='lines',
                    name=model_score_names[i]))
fig.show()

In [ ]:
train_model_input = {name:X_train[name] for name in feature_names}
model = AutoInt(linear_feature_columns,dnn_feature_columns,
               task='binary',dnn_dropout=0.5)
model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['AUC','FalsePositives','FalseNegatives','TruePositives','TrueNegatives'])
model.fit(train_model_input, y_train,batch_size=4096, epochs=2, verbose=0,validation_data=[x_val,y_val])
train_res = model.evaluate(train_model_input,y_train,batch_size=4096)
val_res = model.evaluate(x_val,y_val,batch_size = 4096)
test_res= model.evaluate(test_model_input,y_test,batch_size=4096)
auc_scores= [train_res[1],val_res[1],test_res[1]]
log_loss = [train_res[0],val_res[0],test_res[0]]
model_score_names = ["Baseline Train","Baseline Val","Baseline Test"]

20000/20000 [==============================] - 1s 27us/sample - loss: 0.4149 - AUC: 0.7135 - FalsePositives: 0.0000e+00 - FalseNegatives: 3396.0000 - TruePositives: 0.0000e+00 - TrueNegatives: 16604.0000


In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=[i for i in range(1,TOTAL_ROUNDS+1)], y=train_auc,
                    mode='lines',
                    name='TRAIN AUC'))
fig.add_trace(go.Scatter(x=[i for i in range(1,TOTAL_ROUNDS+1)], y=round_val_auc,
                    mode='lines',
                    name='VAL AUC'))
fig.add_trace(go.Scatter(x=[i for i in range(1,TOTAL_ROUNDS+1)], y=rounds_auc_user_avg,
                    mode='lines',
                    name='TEST AUC'))

for i in range(len(auc_scores)):
  fig.add_trace(go.Scatter(x=[i for i in range(1,TOTAL_ROUNDS+1)], y=[auc_scores[i]]*100,
                    mode='lines',
                    name=model_score_names[i]))
fig.show()

In [ ]:

fig = go.Figure()
fig.add_trace(go.Scatter(x=[i for i in range(1,TOTAL_ROUNDS+1)], y=train_loss,
                    mode='lines',
                    name='TRAIN LOSS'))
fig.add_trace(go.Scatter(x=[i for i in range(1,TOTAL_ROUNDS+1)], y=round_val_loss,
                    mode='lines',
                    name='VAL LOSS'))
fig.add_trace(go.Scatter(x=[i for i in range(1,TOTAL_ROUNDS+1)], y=rounds_loss_user_avg,
                    mode='lines',
                    name='TEST LOSS'))
for i in range(len(log_loss)):
  fig.add_trace(go.Scatter(x=[i for i in range(1,TOTAL_ROUNDS+1)], y=[log_loss[i]]*100,
                    mode='lines',
                    name=model_score_names[i]))
fig.show()

In [ ]:
train_loss =[0.47471890869140626, 0.45933503937721254, 0.45191054611206055, 0.44706381130218503, 0.44384197940826414, 0.44138774909973144, 0.4393663149356842, 0.43749199023246765, 0.4357428717136383, 0.43407392630577085, 0.4324373745918274, 0.43091747136116026, 0.42944961924552916, 0.4278727382183075, 0.4262581744670868, 0.4247138774394989, 0.4229962733268738, 0.4216325581073761, 0.42023342232704164, 0.4187789542198181, 0.41740368723869326, 0.4160254861831665, 0.4149288161277771, 0.41408221888542174, 0.41317824802398684, 0.4124322004318237, 0.41174537477493284, 0.4112931337833405, 0.41076324062347414, 0.41039819831848146, 0.4102805901527405, 0.4098808511734009, 0.4096204180240631, 0.4094522511959076, 0.40928319873809815, 0.409182008934021, 0.4090389958381653, 0.4089897649288177, 0.4088135678291321, 0.4087381363391876, 0.40872506008148196, 0.40872339844703676, 0.40856402707099915, 0.40866189279556275, 0.40840030326843263, 0.4084196773529053, 0.40827814660072326, 0.40831294717788696, 0.40831238985061646, 0.4082072415828705, 0.40837022438049314, 0.40810661630630496, 0.40806673493385315, 0.40804146399497987, 0.4080231209278107, 0.4081127182483673, 0.4080119821548462, 0.407945167350769, 0.40800280017852786, 0.4079904001712799, 0.4079435334205627, 0.4079054205417633, 0.407948850774765, 0.4079104496002197, 0.4079413959980011, 0.4078433696746826, 0.4081219146728516, 0.40785918216705325, 0.40789666547775266, 0.4078794421672821, 0.4080409633636475, 0.4079259019851685, 0.40787105841636656, 0.4078469407081604, 0.4079276698589325, 0.4078082116127014, 0.40785732111930845, 0.40782229437828066, 0.4078802754878998, 0.40784056787490847, 0.4078846857070923, 0.4078979387760162, 0.4078854732513428, 0.4078898064136505, 0.40794833154678345, 0.4078718288898468, 0.40789730639457705, 0.40786352229118344, 0.407902477312088, 0.4079653787612915, 0.40783147139549253, 0.4078165147781372, 0.40778909940719604, 0.4079109533786774, 0.4077943943500519, 0.4080168433666229, 0.4078799606800079, 0.4078231729030609, 0.4077914487361908, 0.40773136506080626]
round_val_loss = [0.4781121024131775, 0.46379222702980044, 0.456368359375, 0.451532154083252, 0.4483344175338745, 0.4460032935142517, 0.44411195154190064, 0.44230029010772703, 0.4407668894767761, 0.43903209619522093, 0.43762558889389036, 0.4361430849552155, 0.4347505188941956, 0.43352468957901, 0.43189551873207094, 0.43071888451576235, 0.4291474023342133, 0.427962038898468, 0.42673384518623353, 0.42555544457435607, 0.4243810737609863, 0.42332804141044617, 0.4224686983585358, 0.4219427380561829, 0.42114612030982973, 0.4208881095409393, 0.42027588510513303, 0.4198391132831574, 0.41984622778892516, 0.41962654457092285, 0.41944054131507874, 0.41948329329490663, 0.4193477464675903, 0.4192911979675293, 0.41913736696243287, 0.4192274349689484, 0.41908201026916503, 0.4190870985507965, 0.41912986426353455, 0.41923641757965086, 0.4191636570453644, 0.41932422671318054, 0.4192918848991394, 0.4193776826381683, 0.419286923456192, 0.4194958628177643, 0.4191586637020111, 0.4195122055053711, 0.4194542636394501, 0.41947180614471435, 0.41936275601387024, 0.41921100721359256, 0.41931993594169614, 0.41915038905143737, 0.419376006603241, 0.41979192843437196, 0.419565066576004, 0.4195216058254242, 0.4195120303153992, 0.41951173763275146, 0.41950710878372194, 0.419581157541275, 0.4197572375774384, 0.41964282054901125, 0.41971590051651003, 0.41972068276405333, 0.4197718028068542, 0.4197851507663727, 0.4196979204654694, 0.4195984719753265, 0.41976308302879334, 0.4197659245967865, 0.41991742730140685, 0.41969116110801696, 0.41976943674087525, 0.41970102396011355, 0.4199143806934357, 0.4198828097820282, 0.4197861261367798, 0.419769621515274, 0.42022134771347047, 0.41997280764579775, 0.4198287165164947, 0.42007475600242616, 0.4200249489307404, 0.4200877274990082, 0.42010667276382446, 0.4199743523597717, 0.42046815428733825, 0.4205467143535614, 0.4203200633525848, 0.4203139139175415, 0.4201621823787689, 0.4200956902503967, 0.420187535905838, 0.4201734440803528, 0.4201800420284271, 0.42004956307411195, 0.4203555142879486, 0.42023239164352416]
rounds_loss_user_avg =[0.4774716869354248, 0.46289758915901186, 0.45536065020561217, 0.45037936525344846, 0.4470381872653961, 0.4444432110309601, 0.4422906684398651, 0.44017969727516176, 0.43835046162605285, 0.4363843417167664, 0.43455758657455446, 0.4327949984550476, 0.4311015558242798, 0.42940114216804504, 0.42756172161102296, 0.42593474807739257, 0.42392931733131406, 0.4224989646434784, 0.4209524865150452, 0.4194335783958435, 0.41787559051513673, 0.41646197748184205, 0.4152843510150909, 0.41437432265281676, 0.4134774374961853, 0.41279402418136596, 0.41215113458633423, 0.4117244225978851, 0.4113022769451141, 0.4109838633060455, 0.4108579478740692, 0.4107124408721924, 0.41044514684677125, 0.4103549579143524, 0.4102180571079254, 0.4102627820968628, 0.41015129709243775, 0.4100938566684723, 0.4100729645729065, 0.4100540269374847, 0.41011128997802737, 0.4101902056217194, 0.4100447649002075, 0.41018795166015626, 0.41004077048301696, 0.4101622645378113, 0.41003040356636045, 0.4101605951309204, 0.41023469009399416, 0.4101220485687256, 0.41032059364318846, 0.41004339280128477, 0.41010836124420164, 0.4100488494873047, 0.4100548182487488, 0.41018634343147276, 0.41004499111175535, 0.41004708518981936, 0.4101254505634308, 0.41008214631080625, 0.410154554605484, 0.4101580774784088, 0.41023662238121034, 0.410274134683609, 0.41032392506599424, 0.41036719379425046, 0.4106143804550171, 0.4104097743034363, 0.41043276176452637, 0.4103933776855469, 0.4106604537963867, 0.4105316427707672, 0.41048202495574954, 0.4104883597373962, 0.4105379393100739, 0.4104404592514038, 0.4104938391685486, 0.41051772866249087, 0.4105815369606018, 0.4105649854183197, 0.41072889008522034, 0.41073118357658384, 0.41074921288490296, 0.4107143570899963, 0.41077066779136656, 0.410679341173172, 0.41069921054840086, 0.4106900549888611, 0.41087705411911013, 0.41096413164138795, 0.41084865760803224, 0.41083864040374757, 0.4108536043167114, 0.4109495828151703, 0.41085530381202695, 0.4110050187110901, 0.41090696849823, 0.4107707524776459, 0.4108586118221283, 0.41080780177116394]
train_auc =[0.51724803, 0.5421857, 0.5618814, 0.5854827, 0.60527545, 0.6189247, 0.6281762, 0.63617325, 0.64154387, 0.64842916, 0.65260935, 0.65789753, 0.6627134, 0.667256, 0.67241037, 0.6759459, 0.68185836, 0.6864306, 0.6904489, 0.69486403, 0.69881076, 0.7026983, 0.7051308, 0.70682764, 0.709134, 0.71096814, 0.71225095, 0.71394145, 0.714463, 0.7151892, 0.71569836, 0.71630883, 0.7167331, 0.71711576, 0.717499, 0.7177354, 0.71807075, 0.71810764, 0.7186328, 0.7187597, 0.7189872, 0.7191966, 0.7193833, 0.7194258, 0.7198671, 0.71991336, 0.7199842, 0.7200899, 0.72036695, 0.72042966, 0.72060066, 0.72053105, 0.720666, 0.7206859, 0.7208384, 0.72064614, 0.7207412, 0.72087234, 0.7208224, 0.7208102, 0.72082037, 0.7209873, 0.7210579, 0.72094184, 0.72096896, 0.7212035, 0.7212636, 0.72111434, 0.72114116, 0.72117066, 0.72115517, 0.7211355, 0.7210964, 0.7211255, 0.7212213, 0.72128373, 0.7210543, 0.72110647, 0.72122383, 0.72102755, 0.721076, 0.72137773, 0.7212517, 0.7211319, 0.72127366, 0.721208, 0.72134596, 0.72129476, 0.72138244, 0.72137094, 0.7212766, 0.7214943, 0.7213862, 0.721568, 0.72145724, 0.72149897, 0.72135305, 0.7214278, 0.7215686, 0.7217177]
round_val_auc =[0.52437425, 0.54928696, 0.5692751, 0.59236807, 0.610554, 0.623874, 0.6324858, 0.63904154, 0.64407986, 0.6505453, 0.6544175, 0.65907407, 0.66333383, 0.6668104, 0.6714099, 0.6745087, 0.6796613, 0.6837195, 0.68693703, 0.6907527, 0.69389117, 0.6969905, 0.69909346, 0.7002349, 0.7020983, 0.70317686, 0.7040746, 0.7050674, 0.70508265, 0.7055496, 0.7059339, 0.70588243, 0.7061095, 0.70641196, 0.7065384, 0.7066294, 0.70671546, 0.7067807, 0.7067449, 0.70674354, 0.70670915, 0.706701, 0.7070261, 0.70678896, 0.70697236, 0.7067151, 0.706996, 0.7066831, 0.70648396, 0.70676684, 0.7067912, 0.707116, 0.7069636, 0.7070189, 0.7068138, 0.7062676, 0.7061522, 0.70631784, 0.7062554, 0.70608425, 0.70610785, 0.70603645, 0.70599854, 0.70592123, 0.7060351, 0.7059429, 0.7057909, 0.70578635, 0.7058233, 0.7059018, 0.70577586, 0.7056873, 0.70596266, 0.7056174, 0.7056484, 0.7055971, 0.70579314, 0.7054957, 0.7055343, 0.7054516, 0.70544004, 0.70534337, 0.7053559, 0.70536405, 0.7051606, 0.7051516, 0.70506585, 0.7050282, 0.7049744, 0.7047293, 0.7046958, 0.7047107, 0.70466447, 0.7049425, 0.70473444, 0.7048906, 0.7045877, 0.7047286, 0.70475936, 0.70461375]
rounds_auc_user_avg =[0.515399, 0.5395777, 0.55918306, 0.5823709, 0.6038276, 0.61858094, 0.6282046, 0.63663024, 0.6425002, 0.6501336, 0.6551691, 0.66105, 0.6660295, 0.67129165, 0.67654246, 0.6802273, 0.6869551, 0.6913658, 0.6955508, 0.6999519, 0.7041774, 0.7079952, 0.71035886, 0.7121894, 0.7143112, 0.7158512, 0.7168907, 0.7183724, 0.7186755, 0.7190987, 0.7197367, 0.71967924, 0.72002053, 0.72010523, 0.7204646, 0.7203601, 0.72039855, 0.7203304, 0.7206356, 0.7205746, 0.72065115, 0.72049373, 0.7209114, 0.72056323, 0.72052765, 0.72046846, 0.7207977, 0.72054935, 0.72060084, 0.72065794, 0.7205168, 0.72072273, 0.7205205, 0.7205595, 0.7206133, 0.7204355, 0.72046083, 0.7204298, 0.7204155, 0.7202966, 0.72022545, 0.7202126, 0.72002375, 0.71999377, 0.71990687, 0.71994007, 0.7197248, 0.71946913, 0.7195281, 0.71968293, 0.7193383, 0.7195612, 0.7194484, 0.71942496, 0.719473, 0.7192589, 0.719231, 0.71911025, 0.719017, 0.7188767, 0.7189106, 0.71879256, 0.71878636, 0.7188762, 0.718608, 0.7187056, 0.7188945, 0.71866155, 0.71888024, 0.71872556, 0.7186202, 0.7185303, 0.7185796, 0.7185556, 0.71847415, 0.71846664, 0.71832275, 0.71836114, 0.7185339, 0.7183769]